In [4]:
import csv
import time
import json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException


In [5]:
with open('./科研机构失败清单.json', encoding='utf-8') as f:
  institu_list = json.load(f)
with open('./data/长三角普通高等学校名单.json', encoding='utf-8') as f:
  school_list = json.load(f)

In [3]:
institu_list[0]

{'科研机构名称': '江苏省环境科学研究院', '研究领域': '环境科学', '省': '江苏省', '市': '南京市', '县/区': '鼓楼区'}

In [15]:
def open_page_author(driver, institu_name, res):
    # 打开知网高级检索页面
    driver.get("https://kns.cnki.net/kns8/AdvSearch?dbprefix=CFLS&&crossDbcodes=CJFQ%2CCDMD%2CCIPD%2CCCND%2CCISD%2CSNAD%2CBDZK%2CCCJD%2CCCVD%2CCJFN")
    # 点击学术期刊栏目
    time.sleep(3)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located(
        (By.XPATH, '/html/body/div[3]/div[1]/div/ul[1]/li[1]/a/span'))).click()
    # 选择作者单位
    time.sleep(3)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@id="gradetxt"]/dd[2]/div[2]/div[1]/div[1]/span'))).click()
    time.sleep(3)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@id="gradetxt"]/dd[2]/div[2]/div[1]/div[2]/ul[2]/li[5]/a'))).click()
    time.sleep(3)
    driver.find_element_by_xpath(
        '//*[@id = "gradetxt"]/dd[2]/div[2]/input').send_keys(institu_name)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@id="gradetxt"]/dd[2]/div[2]/div[2]/div/span'))).click()
    time.sleep(3)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@id="gradetxt"]/dd[2]/div[2]/div[2]/ul/li[1]/a'))).click()
    time.sleep(3)
    # 选择年份
    WebDriverWait(driver, 100).until(EC.presence_of_element_located(
        (By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div/input'))).click()
    time.sleep(3)
    time0_element = driver.find_element_by_xpath(
        '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/ul')
    driver.execute_script(
        "arguments[0].scrollTop = arguments[0].scrollHeight", time0_element)
    time.sleep(3)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located(
        (By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/ul/li[45]/a'))).click()
    time.sleep(3)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located(
        (By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/div/input'))).click()
    time.sleep(3)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located(
        (By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/ul/li[3]/a'))).click()
    time.sleep(3)

    # 点击检索
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[2]/input'))).click()
    time.sleep(10)

    while True:
        # 找到table中的所有tr元素
        WebDriverWait(driver, 100).until(EC.presence_of_element_located(
            (By.XPATH, '/html/body/div[3]/div[2]/div[2]/div[2]/form/div/table')))
        time.sleep(3)
        table = driver.find_element_by_xpath(
            '/html/body/div[3]/div[2]/div[2]/div[2]/form/div/table')
        WebDriverWait(driver, 100).until(EC.presence_of_element_located(
            (By.XPATH, './/tr')))
        time.sleep(3)
        rows = table.find_elements_by_xpath('.//tr')

        # 遍历每个tr元素
        for row in rows[1:]:
            # 找到该行中的所有td元素
            cells = row.find_elements_by_xpath('.//td')
            article = {}
            # 遍历每个td元素，并将它们的文本值存储在一个字典对象中
            article['单位'] = institu_name
            article['名称'] = cells[1].text
            article['作者'] = cells[2].text
            article['刊名'] = cells[3].text
            article['时间'] = cells[4].text
            res.append(article)

        try:
            # # 找到下一页元素，并单击它
            # next_page = driver.find_element_by_css_selector('#PageNext')
            # next_page.click()

            # # 等待页面加载完成
            # driver.implicitly_wait(10)
            WebDriverWait(driver, 20).until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, '#PageNext'))).click()
            time.sleep(3)
            # 找到table元素
        except TimeoutException:
            print(institu_name + ' 获取数据超时')
            # 如果没有下一页数据，则退出循环
            break
        except Exception as e:
            break
    # 循环获取每一页的数据

[{'单位': '江苏省建筑科学研究院有限公司', '名称': '标准化附框在隔热铝合金外窗防渗漏施工中的应用', '作者': '沈佑竹; 刘永刚; 魏燕丽', '刊名': '江苏建筑', '时间': '2021-12-31'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '江苏省居住建筑节能75%技术研究与应用', '作者': '魏燕丽; 许锦峰', '刊名': '新型建筑材料', '时间': '2021-12-25'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '金鸡湖隧道现浇侧墙结构抗裂性能提升技术与应用', '作者': '张坚; 徐文; 王育江; 康健; 贺飞', '刊名': '江苏建筑', '时间': '2021-10-31'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '大跨钢桁架屋盖结构选型及节点抗震分析', '作者': '肖蔚; 沈古成; 缪汉良', '刊名': '江苏建筑', '时间': '2021-10-31'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '大直径钢管内无收缩高抗裂混凝土设计与制备', '作者': '颜智法; 董香港; 杨睿; 李明; 徐文', '刊名': '江苏建筑', '时间': '2021-10-31'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '钢框架结构检测鉴定与加固设计关键分析', '作者': '骆利', '刊名': '工程建设与设计', '时间': '2021-10-28'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '摇摆钢支撑快速建筑信息模型建模技术及其Revit模块开发', '作者': '卞明月;陈鑫;谈丽华;刘涛;孙勇', '刊名': '科学技术与工程', '时间': '2021-09-18'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': 'BIM+装配式标准化设计在住宅项目中的研究及应用', '作者': '高威; 宣云干; 孙俊; 陶在来', '刊名': '江苏建筑', '时间': '2021-08-31'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '绿色保障性住房关键技术研究与应用示范', '作者': '刘永刚;路宏伟;刘建石;许锦

In [17]:
len(result)

22

In [12]:
if __name__ == "__main__":
    # get直接返回，不再等待界面加载完成
    desired_capabilities = DesiredCapabilities.CHROME
    desired_capabilities["pageLoadStrategy"] = "none"

    # 设置谷歌驱动器的环境
    options = webdriver.ChromeOptions()
    # 设置chrome不加载图片，提高速度
    options.add_experimental_option(
        "prefs", {"profile.managed_default_content_settings.images": 2})
    # # 设置不显示窗口
    # options.add_argument('--headless')
    # 创建一个谷歌驱动器
    driver = webdriver.Chrome(options=options)
    result = []
    # institu_list = json.loads(open('./长三角科研机构知网论文数.json').read())
    open_page_author(driver, "江苏省建筑科学研究院有限公司", result)
    # for institu in institu_list:
    #     if '中文文献数' in institu:
    #         open_page_author(driver, institu['科研机构名称'], result)
    # with open('./长三角论文.json', 'w', encoding='utf-8') as file:
    #     json.dump(result, file)
    driver.close()


[{'单位': '江苏省建筑科学研究院有限公司', '名称': '标准化附框在隔热铝合金外窗防渗漏施工中的应用', '作者': '沈佑竹; 刘永刚; 魏燕丽', '刊名': '江苏建筑', '时间': '2021-12-31'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '江苏省居住建筑节能75%技术研究与应用', '作者': '魏燕丽; 许锦峰', '刊名': '新型建筑材料', '时间': '2021-12-25'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '金鸡湖隧道现浇侧墙结构抗裂性能提升技术与应用', '作者': '张坚; 徐文; 王育江; 康健; 贺飞', '刊名': '江苏建筑', '时间': '2021-10-31'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '大跨钢桁架屋盖结构选型及节点抗震分析', '作者': '肖蔚; 沈古成; 缪汉良', '刊名': '江苏建筑', '时间': '2021-10-31'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '大直径钢管内无收缩高抗裂混凝土设计与制备', '作者': '颜智法; 董香港; 杨睿; 李明; 徐文', '刊名': '江苏建筑', '时间': '2021-10-31'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '钢框架结构检测鉴定与加固设计关键分析', '作者': '骆利', '刊名': '工程建设与设计', '时间': '2021-10-28'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '摇摆钢支撑快速建筑信息模型建模技术及其Revit模块开发', '作者': '卞明月;陈鑫;谈丽华;刘涛;孙勇', '刊名': '科学技术与工程', '时间': '2021-09-18'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': 'BIM+装配式标准化设计在住宅项目中的研究及应用', '作者': '高威; 宣云干; 孙俊; 陶在来', '刊名': '江苏建筑', '时间': '2021-08-31'}, {'单位': '江苏省建筑科学研究院有限公司', '名称': '绿色保障性住房关键技术研究与应用示范', '作者': '刘永刚;路宏伟;刘建石;许锦

TimeoutException: Message: 


In [21]:
result = {}
for school in school_list:
  if '中文文献数' not in school:
    continue
  if school['城市'] in result:
    result[school['城市']] += school['中文文献数']
  else:
    result[school['城市']] = school['中文文献数']

import csv

# 创建 CSV 文件并写入数据
with open('result.csv', 'w', encoding='gbk', newline='') as f:
    writer = csv.writer(f)

    # 写入 CSV 文件的标题行
    writer.writerow(['城市', '知网中文文献数'])

    # 遍历字典的列表，并将每个字典的数据写入 CSV 文件中
    for item in result.keys():
        row = [item, result[item]]
        writer.writerow(row)


In [32]:
import json 

with open('./长三角科研机构知网论文数.json', encoding='utf-8') as file:
  data = json.load(file)

for i in data:
  if '中文文献数' not in i:
    i['中文文献数'] = 0

with open('./长三角科研机构知网论文数.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False)
